In [71]:
import os
import json
from collections import OrderedDict

import torch
from torch import nn
from torch import optim
from tqdm.notebook import tqdm

from torch.utils.data import Dataset, DataLoader
from typing import Dict, List, Tuple, Any

import numpy as np
from matplotlib import pyplot as plt

Connect to google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1) Load the data

In [72]:
data_dir = '/content/drive/My Drive/datasets/2009-skoltech-hack/data'

# read the data
data_path = os.path.join(data_dir, 'train_data_scaled.json')
with open(data_path, 'r') as json_file:
    data = json.load(json_file)

# read the labels
labels_path = os.path.join(data_dir, 'train_labels.json')
with open(labels_path, 'r') as json_file:
    labels = json.load(json_file)

print('Loaded')

Loaded


## 2) Define Dataset and DataLoader

In [73]:
product_counts = np.array([7418, 30983,  7375, 32560, 29704,  2652,  7720], dtype=np.float)
product_probs = 1 / (product_counts**(1.5))
product_probs /= np.sum(product_probs)

class ProductDataset(Dataset):
    def __init__(self, 
                 data: Dict[int, List[Any]], 
                 labels: Dict[int, List[int]]):
        """
        data: Dict with structure: {client_id: List of features}
        labels: Dict with structure: {client_id: List with 7 zeros or ones}
        """
        super(ProductDataset, self).__init__()
        self.data = data
        self.labels = labels
        
        self.ix_to_key = dict()
        
        for ix, key in enumerate(self.data):
            self.ix_to_key[ix] = key
            

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        cust_id = self.ix_to_key[idx] # get customer id
        
        products = np.zeros(7, dtype=np.float32)
        for label_ix, product_status in enumerate(self.labels[cust_id]):
            if product_status:
                products[label_ix] = 1
                
        # randomly close one product
        ones_ixs = np.where(products == 1)[0]
        ones_probs = product_probs[ones_ixs]
        ones_probs /= np.sum(ones_probs)
        chosen_ix = np.random.choice(ones_ixs, replace=False, p = ones_probs)
        
        # zero that position
        products[chosen_ix] = 0
        
        # form one-hot target
        targets = np.zeros_like(products, dtype=np.float32)
        targets[chosen_ix] = 1
        
        item = {
            "features": {},
            "targets": targets,
        }

        features = np.array(self.data[cust_id])
        features = np.hstack((features, products))
        
        item["features"] = torch.from_numpy(features).float()


        return item

In [74]:
BATCH_SIZE = 16

product_dataset = ProductDataset(data=data, 
                                 labels=labels)

product_dataloader = DataLoader(dataset=product_dataset, 
                                batch_size = BATCH_SIZE, 
                                shuffle=True, 
                                num_workers = 4)

In [75]:
product_dataset[2]['features'].shape

torch.Size([169])

## 3) Define the model

In [76]:
params = {
    'input_dim': 162+7,
    'emb_dim': 64,
    'transformer_nhead': 2,
    'transformer_dim_feedforward': 32,
    'transformer_dropout': 0.1,
    'dense_unit': 128,
    'num_layers': 2,
    'n_products': 7
}

In [77]:
class ProductModel(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Let's say embedding
        self.embedding = nn.Linear(in_features=params['input_dim'], 
                                   out_features=params["emb_dim"])

        transformer_blocks = []
        for i in range(params["num_layers"]):
            transformer_block = nn.TransformerEncoderLayer(
                d_model=params["emb_dim"],
                nhead=params["transformer_nhead"],
                dim_feedforward=params["transformer_dim_feedforward"],
                dropout=params["transformer_dropout"],
            )
            transformer_blocks.append(
                (f"transformer_block_{i}", transformer_block)
            )

        self.transformer_encoder = nn.Sequential(
            OrderedDict(transformer_blocks)
        )

        self.linear = nn.Linear(
            in_features=params["emb_dim"], out_features=params["dense_unit"]
        )
        self.scorer = nn.Linear(
            in_features=params["dense_unit"],
            out_features=params["n_products"],
        )

    def forward(self, features):

        emb_features = self.embedding(features).unsqueeze(1)

        transformer_output = self.transformer_encoder(emb_features)
        pooling = torch.mean(transformer_output, dim=1)
        linear = torch.tanh(self.linear(pooling))
        merch_logits = self.scorer(linear)

        return merch_logits

### Sanity check

In [78]:
model = ProductModel(params=params)

criterion = nn.BCEWithLogitsLoss()
batch = next(iter(product_dataloader))
output = model(batch['features'])

loss = criterion(output, batch['targets'])
print(loss)

tensor(0.6913, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


## 4) Train loop without Catalyst

In [79]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [80]:
def calculate_metrics(logits: torch.tensor, 
                      targets: torch.tensor, 
                      num_classes: int = 7):
    """
    Calculates position-wise count matched
    """

    logits = logits.detach().cpu().numpy()
    prediction = np.argmax(logits, axis=1)
    prediction_one_hot = np.zeros((prediction.size, num_classes))
    prediction_one_hot[np.arange(prediction.size), prediction] = 1
    prediction_one_hot = prediction_one_hot.astype(bool)

    targets = targets.detach().cpu().numpy().astype(bool)

    matches = np.logical_and(prediction_one_hot, targets).astype(int)
    count_matches = np.sum(matches, axis=0)
    
    return count_matches

In [81]:
def train(model, 
          train_dataloader, 
          criterion,
          optimizer,
          device: str = 'cuda', 
          num_epochs: int = 10, 
          num_classes: int = 7, 
          eps = 1e-6):
    """
    model: torch nn.Module
    train_dataloader: torch DataLoader
    """

    all_losses = []
    all_counts = []

    model = model.to(device)
    best_mean_acc = 0
    for epoch in range(num_epochs):
        running_loss = 0
        
        num_samples_per_epoch = np.zeros(num_classes, dtype=np.float64) + eps
        class_wise_counts = np.zeros(num_classes, dtype = int) 
        for batch in tqdm(train_dataloader):
            # train body
            optimizer.zero_grad()

            features = batch['features'].to(device)
            targets = batch['targets'].to(device)

            logits = model(features)

            loss = criterion(logits, batch['targets'].to(device))
            loss.backward()

            
            optimizer.step()

            # save metrics
            with torch.no_grad():
                running_loss += loss.item()
                class_wise_counts += calculate_metrics(logits=logits, targets=targets)

                all_losses.append(loss.item())
                
                num_samples_per_epoch += targets.detach().cpu().numpy().sum(axis=0)
                all_counts.append(class_wise_counts / num_samples_per_epoch)

        # epoch metrics

        running_loss /= len(train_dataloader)
        
        
        class_wise_counts = class_wise_counts.astype(np.float64) + eps
        class_wise_counts /= num_samples_per_epoch
        mean_acc = np.mean(class_wise_counts)

        print(f'EPOCH: {epoch + 1}')
        print(f'### BSE loss: {running_loss:.5f}')
        print('#### num samples: ', '\t'.join([str(round(elem)) for elem in num_samples_per_epoch]))
        print(f'###### Accuracy:', '\t'.join([str(round(elem, 5)) for elem in class_wise_counts]), f'Mean: {mean_acc:.4f}')
        print()
        if mean_acc > best_mean_acc:
          best_mean_acc = mean_acc
          torch.save(model.state_dict(), os.path.join(data_dir, f"prod_scaled_epoch_{epoch+1}_ckpt.pth"))

    return model, all_losses, all_counts


In [82]:
model = ProductModel(params=params)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 3*1e-4)

trained_model, losses, counts = train(model=model, 
                                      train_dataloader=product_dataloader, 
                                      criterion=criterion, 
                                      optimizer=optimizer, 
                                      device=device, 
                                      num_epochs=100)


EPOCH: 1
### BSE loss: 0.29939
#### num samples:  4669.0	12390.0	4613.0	10358.0	9779.0	2336.0	5280.0
###### Accuracy: 0.04348	0.68709	0.24865	0.51458	0.71531	0.15111	0.23352 Mean: 0.3705




EPOCH: 2
### BSE loss: 0.25048
#### num samples:  4681.0	12450.0	4543.0	10483.0	9644.0	2335.0	5289.0
###### Accuracy: 0.1019	0.66803	0.31125	0.61652	0.90087	0.28522	0.35394 Mean: 0.4625




EPOCH: 3
### BSE loss: 0.24485
#### num samples:  4729.0	12362.0	4527.0	10359.0	9881.0	2333.0	5234.0
###### Accuracy: 0.15162	0.66745	0.33643	0.62014	0.89991	0.28847	0.35709 Mean: 0.4744




EPOCH: 4
### BSE loss: 0.24104
#### num samples:  4692.0	12403.0	4584.0	10350.0	9792.0	2352.0	5252.0
###### Accuracy: 0.16837	0.68008	0.35231	0.63488	0.89277	0.29932	0.36101 Mean: 0.4841




EPOCH: 5
### BSE loss: 0.23830
#### num samples:  4761.0	12436.0	4493.0	10390.0	9797.0	2356.0	5192.0
###### Accuracy: 0.19513	0.69001	0.35299	0.64167	0.88384	0.31367	0.36017 Mean: 0.4911



...


EPOCH: 96
### BSE loss: 0.19761
#### num samples:  4666.0	12477.0	4570.0	10425.0	9721.0	2307.0	5259.0
###### Accuracy: 0.32726	0.78032	0.48337	0.74926	0.8783	0.51539	0.45788 Mean: 0.5988




EPOCH: 97
### BSE loss: 0.19699
#### num samples:  4692.0	12360.0	4559.0	10511.0	9711.0	2332.0	5260.0
###### Accuracy: 0.32609	0.78519	0.47642	0.74246	0.88199	0.52444	0.46673 Mean: 0.6005




EPOCH: 98
### BSE loss: 0.19713
#### num samples:  4668.0	12492.0	4511.0	10403.0	9753.0	2349.0	5249.0
###### Accuracy: 0.32562	0.78202	0.47639	0.74402	0.8785	0.52107	0.46504 Mean: 0.5990




EPOCH: 99
### BSE loss: 0.19748
#### num samples:  4699.0	12472.0	4571.0	10440.0	9682.0	2318.0	5243.0
###### Accuracy: 0.32369	0.77983	0.47714	0.74033	0.87565	0.5289	0.46176 Mean: 0.5982




EPOCH: 100
### BSE loss: 0.19756
#### num samples:  4699.0	12385.0	4620.0	10481.0	9659.0	2362.0	5219.0
###### Accuracy: 0.32432	0.78571	0.4842	0.73953	0.88674	0.53514	0.46273 Mean: 0.6026

